In [1]:
import numpy as np
import torch
from torch.utils import data

In [2]:
def synthetic_data(w, b, num_examples):
    '''
    generate random data
    y = Xw + b + epsilon
    '''
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

# generate data
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [5]:
# data pipeline
def load_array(data_arrays, batch_size, shuffle=True):
    '''
    构造 pytorch 数据迭代器
    '''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=shuffle)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [7]:
# model definition
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 输入维度 2, 输出维度 1

# init model param
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

net[0].weight, net[0].bias

(Parameter containing:
 tensor([[-0.0059,  0.0098]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True))

In [8]:
# define loss & trainer
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [10]:
# train
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000308
epoch 2, loss 0.000098
epoch 3, loss 0.000098
epoch 4, loss 0.000100
epoch 5, loss 0.000098
epoch 6, loss 0.000098
epoch 7, loss 0.000099
epoch 8, loss 0.000099
epoch 9, loss 0.000100
epoch 10, loss 0.000098
